In [1]:
# Web Scraping
from bs4 import BeautifulSoup
from lxml import html
import requests

import random
import re
import os
import pandas as pd
import numpy as np

# 1) Web Scraping Google News

Because google.com is so sensitive to web scraping, this became a very manual process.
I scraped google news for 18 specify news outlets for two months on HK protest related headlines+Description.

For each news outlets:
- seach words: protest protests protester protesters + "hong kong" site: news outlet websit
- set news date range 7/1 - 7/31 and 8/1 - 8/31
- Copy the url for result page to url cell below
- build a panads dataframe for each new outlet and pickle it.

In [2]:
# Function to read html, scape the data, and form dataframe
def scape_page(url):
    user_agent_list = [
       #Chrome
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
         ]
    
    url = url+"&num=100" #show 100 headlines on the page
    headers = {"User-Agent":random.choice(user_agent_list)}
    response = requests.get(url,headers=headers)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    publishers = []
    titles = []
    intros = []
    links = []
    dates = []  
    # Append data to list
    publishers += [publisher.text for publisher in soup.find_all('div', attrs = {'class': 'pDavDe RGRr8e'})]
    titles += [title.text for title in soup.find_all('div', attrs = {'class': 'phYMDf nDgy9d'})]
    intros += [intro.text for intro in soup.find_all('div', attrs = {'class': 'eYN3rb'})]
    links += [link.get('href') for link in soup.find_all('a', style="text-decoration:none;display:block")]
    dates += [date.text for date in soup.find_all('span', attrs = {'class':"eNg7of"})]

    # Turn data into DataFrame

    news_dict = {'publisher':publishers,'title':titles,'intro':intros,'link':links,'date':dates}
    news_dict_df = pd.DataFrame(news_dict)
    return news_dict_df

In [3]:
# Create a new variable for each news outlet
wsj_df = pd.DataFrame()

In [4]:
# update the url for each each seach result page
url = 'https://www.google.com/search?biw=1920&bih=1066&tbs=cdr%3A1%2Ccd_min%3A7%2F1%2F2019%2Ccd_max%3A8%2F9%2F2019&tbm=nws&sxsrf=ACYBGNQlIPnGPOcbx8VtOshPF4hPgTm8JQ%3A1568087465789&ei=qR13XbfqL4KY_QaI-qeYCg&q=protest%7Cprotests%7Cprotester%7Cprotesters+%22hong+kong%22+site%3Awsj.com&oq=protest%7Cprotests%7Cprotester%7Cprotesters+%22hong+kong%22+site%3Awsj.com&gs_l=psy-ab.12...0.0.0.17425.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.OXoGj9YrMW0'

In [5]:
# run the function above and create a temporarily dataframe to hold the new data from web.
temp_df = scape_page(url)
temp_df.shape

200


(100, 5)

In [6]:
# Append the temp dataframe into the news outlet dataframe
wsj_df = wsj_df.append(temp_df, ignore_index=True)
wsj_df.shape

(100, 5)

In [7]:
# Check the dataframe
wsj_df.head(5)

,publisher,title,intro,link,date
0,Wall Street Journal,Hong Kong Protests Boil Over After Activists O...,HONG KONG—Protesters smashed wall-size windows...,https://www.wsj.com/articles/hong-kong-braces-...,"Jul 2, 2019"
1,Wall Street Journal,Parts of Hong Kong Paralyzed by Protests After...,Parts of Hong Kong Paralyzed by Protests After...,https://www.wsj.com/articles/hong-kong-protest...,"Jul 28, 2019"
2,Wall Street Journal,"China Warns Hong Kong Protesters, Signals Grow...",BEIJING—China's top office for Hong Kong affai...,https://www.wsj.com/articles/china-vows-to-pun...,1 month ago
3,Wall Street Journal,China Claims U.S. ‘Black Hand’ Is Behind Hong ...,China ratcheted up its accusations of U.S. inv...,https://www.wsj.com/articles/china-claims-u-s-...,1 month ago
4,Wall Street Journal,China’s State Media Show Hong Kong Protest Ima...,China's State Media Show Hong Kong Protest Ima...,https://www.wsj.com/articles/china-lets-hong-k...,"Jul 22, 2019"


In [8]:
# pickle the dataframe for each news outlet
wsj_df.to_pickle('raw_data/wsj_us')

# 2) NewsAPI

Secondary data source for headlines and intro if Google News is not showing enough amount of results.

In [8]:
import pandas as pd
import numpy as np
from newsapi import NewsApiClient
api_key='you api key here'
newsapi = NewsApiClient(api_key='api_key')

In [3]:
#To check what news sources the api includes.
#print(newsapi.get_sources())

In [9]:
#q='(protest OR protests OR protester OR protesters)+"hong kong"'
# call API
all_articles = newsapi.get_everything(q='(protest OR protests OR protester OR protesters)+"hong kong"',
                                      sources='the-wall-street-journal',
                                      #domains='https://news.google.com',
                                      from_param='2019-08-25',
                                      to='2019-09-20',
                                      language='en',
                                      sort_by='publishedAt',
                                      page_size=100,
                                      page=None)

In [14]:
all_articles['totalResults']

34

In [15]:
def news_table(all_articles):
    """Returns dataframe using the API result"""
    publishers = []
    titles = []
    intros = []
    links = []
    dates = [] 
    for a in all_articles['articles']:
        publishers.append(a['source']['name'])
        titles.append(a['title'])
        intros.append(a['description'])
        links.append(a['url'])
        dates.append(a['publishedAt'])
    news_dict = {'publisher':publishers,'title':titles,'intro':intros,'link':links,'date':dates}
    news_dict_df = pd.DataFrame(news_dict)
    return news_dict_df

In [16]:
wsj_df = news_table(all_articles)

In [17]:
wsj_df.head()

,publisher,title,intro,link,date
0,The Wall Street Journal,'You Don't Have to Face It Alone.' Hong Kong P...,'You Don't Have to Face It Alone.' Hong Kong P...,https://www.wsj.com/articles/you-dont-have-to-...,2019-09-20T14:32:00Z
1,The Wall Street Journal,FedEx Pilot Detained by Police in Southern Chi...,FedEx Pilot Detained by Police in Southern Chi...,https://www.wsj.com/articles/fedex-pilot-detai...,2019-09-19T15:26:00Z
2,The Wall Street Journal,Bud Brewer Takes Another Shot at Asian IPO Aft...,Bud Brewer Takes Another Shot at Asian IPO Aft...,https://www.wsj.com/articles/budweiser-brewer-...,2019-09-17T11:37:00Z
3,The Wall Street Journal,U.S. Stocks Retreat as Oil Prices Surge - The ...,U.S. Stocks Retreat as Oil Prices Surge The Wa...,https://www.wsj.com/articles/u-s-stock-futures...,2019-09-16T13:38:00Z
4,The Wall Street Journal,How Hong Kong Protesters Evade Authorities Wit...,Protesters in Hong Kong fear they are being mo...,https://www.wsj.com/video/how-hong-kong-protes...,2019-09-16T09:30:00Z


In [18]:
wsj_df.shape

(34, 5)

In [19]:
#Combine with the wsj pickle from google news
wsj_df = wsj_df.append(pd.read_pickle('datapickle/wsj_us'), ignore_index=True)

In [20]:
wsj_df.to_pickle('datapickle/wsj_us')